### Required Libraries

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from functions import decile_analysis
print('********* Required Libraries Imported *********')

********* Required Libraries Imported *********


### Load Dataset

In [2]:
#load model developing data
model_data = pd.read_parquet('Leobrix_Release_BBILAP_Combined_Data.parquet.gzip')
print('********* Data loaded ********* ')
print(model_data.shape)
# Make sure no duplicates in the data
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
print(model_data.shape)

********* Data loaded ********* 
(478328, 2831)
(478328, 2831)


In [3]:
model_data.shape

(478328, 2831)

In [4]:
model_data.shape

(478328, 2831)

In [5]:
def map_weight(value):
    if (value >= '2022-10') and (value <= '2023-03'):
        return 2
    elif (value >= '2021-01') and (value <= '2021-06'):
        return 0.5
    else:
        return 1

model_data['Weight_vintage'] = model_data['month_year'].apply(map_weight)


### Drop Null target records

In [6]:
model_data['60_in_15mob'].value_counts(dropna=False)

60_in_15mob
NaN    354961
0.0    109935
1.0     13432
Name: count, dtype: int64

In [7]:
# Drop Null values of Target 
df = model_data.dropna(subset=['60_in_15mob'])
# take one copy for model development data
data = df.copy()
#some bank data have inf very rate but by safe replace 
data.replace([np.inf, -np.inf], np.nan, inplace=True)

In [8]:
data.shape

(123367, 2832)

### Split the Time interval

In [9]:
model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2023-03')]
oot_df1= data[(data['month_year']>='2023-04') & (data['month_year']<='2023-06')]

In [10]:
model_df1.shape, oot_df1.shape

((101984, 2832), (17343, 2832))

In [14]:
features_list = [
#Bureau Intutive
    
'num_PERSONAL LOAN_enq_last_6M',
'num_CONSUMER LOAN_enq_last_6M_36M',
'Num_of_PERSONAL LOAN_opened_last6M_12M',
'num_CREDIT CARD_enq_last_12M',
'Utilization_all',
'Total_outstanding_Balance_CREDIT CARD_opened_last_9M',
'highest_sanction_amount_credit_cards',
'Age_of_oldest_open_account',
'Utilization_CREDIT CARD_opened_last_36M',
'total_emi',
'total_credit_all_open',
    
# banking Intutive
'bounced_transactions_count',
'Min_Balance_Min_M1_M2_M3',
'num_credit_gt_salary',
'upi_trans_count',
'Max_Credit_Avg_M1_M2_M3',
'ratio_total_debit_to_total_credit_M3',

#new banking features
'ratio_balance_after_2_days_max_credit_M3',
'ratio_5th_day_balance_to_max_credit_M3',
                 
#Loan application Features
'deviation_from_median_income/standard_deviation_income#last_365_Days',
'loan_amount_to_income#last_365_Days',
    
    
#ID Features
'aggs907',
'dm216s',
'dm206s',
'balmag01',
'g406s',

# New BUreau features
'percentage_ever_delinquent', 
'months_since_most_recent_delinquency_installment', 
'months_since_most_recent_delinquency_all',
'percentage_utilization_gt_75',
'No of enq in past 90days',
'Num_Enq_non_BANK_last_12M',
'Num_of_installment_loan_opened',

#PG synthetic Features
'distinct_merchant',
'MEAN(raw.amount WHERE device_os_category = Windows)#ever',
'days_on_pg',
'NUM_UNIQUE(pg_success_raw_data.merchant_id WHERE revised_merchant_category = marketplace)',
'amt_first_android_txn', 
'Weight_vintage',
'month_year',
'60_in_15mob']

In [15]:
len(features_list)

41

In [16]:
# Selected Features for model build purpose
model_df = model_df1[features_list]

In [17]:
oot_df = oot_df1[features_list]

In [18]:
model_df.shape, oot_df.shape

((101984, 41), (17343, 41))

In [19]:
model_df['60_in_15mob'].value_counts(dropna=False)

60_in_15mob
0.0    90856
1.0    11128
Name: count, dtype: int64

In [20]:
oot_df['60_in_15mob'].value_counts(dropna=False)

60_in_15mob
0.0    15505
1.0     1838
Name: count, dtype: int64

In [21]:
# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', '60_in_15mob']) # Adjust 'target' to your actual target column name
y_model = model_df['60_in_15mob']

In [22]:
X_oot = oot_df.drop(columns=['month_year', '60_in_15mob'])
y_oot = oot_df['60_in_15mob']

In [23]:
#Handling Null Values
for i in X_model.columns:
    X_model[i].fillna(-9999,inplace=True)

/tmp/ipykernel_3031711/2233446063.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_model[i].fillna(-9999,inplace=True)


In [24]:
for i in X_oot.columns:
    X_oot[i].fillna(-9999,inplace=True)

/tmp/ipykernel_3031711/1979277635.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_oot[i].fillna(-9999,inplace=True)


In [25]:
X_oot.shape,y_oot.shape

((17343, 39), (17343,))

### Split Model dataset into Train and Test

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.40,random_state=42)

In [27]:
#Split Training data into 80% train and 20% eval
X_train_new, X_eval, y_train_new, y_eval = train_test_split(
    X_train, y_train, test_size=0.2, random_state =42
)
print(X_train_new.shape, X_eval.shape, X_test.shape)

(48952, 39) (12238, 39) (40794, 39)


In [28]:
X_train.shape, X_test.shape

((61190, 39), (40794, 39))

In [29]:
# Get Vinateg weight values 

train_weights = X_train_new['Weight_vintage'].values
eval_weights = X_eval['Weight_vintage'].values
test_weights = X_test['Weight_vintage'].values


In [30]:
oot_weights = X_oot['Weight_vintage'].values

In [31]:
#  remove weight vintage from feature 
X_train_new = X_train_new.drop(columns=['Weight_vintage'])

In [32]:
X_eval = X_eval.drop(columns=['Weight_vintage'])

In [34]:
X_test = X_test.drop(columns=['Weight_vintage'])
X_oot = X_oot.drop(columns=['Weight_vintage'])

In [35]:
# 2. **Prepare DMatrix for XGBoost**
dtrain = xgb.DMatrix(X_train_new, label=y_train_new, weight=train_weights)
deval = xgb.DMatrix(X_eval, label=y_eval,weight=eval_weights)
dtest = xgb.DMatrix(X_test, label=y_test)
doot = xgb.DMatrix(X_oot, label=y_oot)


In [37]:
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'learning_rate': 0.01,
     'max_depth': 2,
    'subsample': 0.6,
    'colsample_bytree': 0.6,

    'lambda': 7,
    'alpha': 4.6,
    'gamma': 5.2,
   
    'scale_pos_weight': 4,
    'random_state': 42
    
}


evals = [(dtrain, 'train'), (deval, 'eval')]
xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round= 1000,
    early_stopping_rounds= 30,
    evals=evals,
    verbose_eval=10
)

X_train_pred = X_train_new.copy()
X_eval_pred = X_eval.copy()
X_test_pred = X_test.copy()
X_oot_pred = X_oot.copy()


X_train_pred['y_actual'] = y_train_new
X_train_pred['y_pred'] = xgb_model.predict(dtrain)
X_train_pred['LID'] = model_df1.loc[X_train_new.index,'latest_feature_id'].values

X_eval_pred['y_actual'] = y_eval
X_eval_pred['y_pred'] = xgb_model.predict(deval)
X_eval_pred['LID'] = model_df1.loc[X_eval.index,'latest_feature_id'].values

X_test_pred['y_actual'] = y_test
X_test_pred['y_pred'] = xgb_model.predict(dtest)
X_test_pred['LID'] = model_df1.loc[X_test.index,'latest_feature_id'].values

X_oot_pred['y_actual'] = y_oot
X_oot_pred['y_pred'] = xgb_model.predict(doot)
X_oot_pred['LID'] = oot_df1.loc[X_oot.index,'latest_feature_id'].values

[0]	train-auc:0.63735	eval-auc:0.62259
[10]	train-auc:0.67412	eval-auc:0.65582
[20]	train-auc:0.67686	eval-auc:0.66070
[30]	train-auc:0.67727	eval-auc:0.66173
[40]	train-auc:0.68070	eval-auc:0.66472
[50]	train-auc:0.68102	eval-auc:0.66492
[60]	train-auc:0.68220	eval-auc:0.66578
[70]	train-auc:0.68275	eval-auc:0.66622
[80]	train-auc:0.68439	eval-auc:0.66808
[90]	train-auc:0.68565	eval-auc:0.66918
[100]	train-auc:0.68606	eval-auc:0.66995
[110]	train-auc:0.68752	eval-auc:0.67055
[120]	train-auc:0.68859	eval-auc:0.67152
[130]	train-auc:0.68912	eval-auc:0.67217
[140]	train-auc:0.68980	eval-auc:0.67222
[150]	train-auc:0.69095	eval-auc:0.67308
[160]	train-auc:0.69134	eval-auc:0.67326
[170]	train-auc:0.69189	eval-auc:0.67374
[180]	train-auc:0.69266	eval-auc:0.67415
[190]	train-auc:0.69344	eval-auc:0.67479
[200]	train-auc:0.69418	eval-auc:0.67540
[210]	train-auc:0.69469	eval-auc:0.67585
[220]	train-auc:0.69530	eval-auc:0.67637
[230]	train-auc:0.69577	eval-auc:0.67700
[240]	train-auc:0.69646	eva

In [38]:
# 6. **Run Decile Analysis**
train_summary, train_auc, train_gini, train_ks = decile_analysis(X_train_pred, 'y_pred', 'y_actual')
eval_summary, eval_auc, eval_gini, eval_ks = decile_analysis(X_eval_pred, 'y_pred', 'y_actual')

test_summary, test_auc, test_gini, test_ks = decile_analysis(X_test_pred, 'y_pred', 'y_actual')
oot_summary, oot_auc, oot_gini, oot_ks = decile_analysis(X_oot_pred, 'y_pred', 'y_actual')

# 7. **Print Results**
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.2f}, KS: {train_ks:.2f}")
print(f"Eval AUC: {eval_auc:.2f}, Gini: {eval_gini:.2f}, KS: {eval_ks:.2f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.2f}, KS: {test_ks:.2f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.2f}, KS: {oot_ks:.2f}")


Train AUC: 0.72, Gini: 0.44, KS: 0.32
Eval AUC: 0.69, Gini: 0.38, KS: 0.29
Test AUC: 0.70, Gini: 0.40, KS: 0.29
OOT AUC: 0.71, Gini: 0.42, KS: 0.31


In [39]:
oot_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.287032         0.583206        0.848376        0.493143   1735   
9          0.185121         0.443725        0.493135        0.402481   1734   
8          0.129758         0.371029        0.402429        0.342330   1734   
7          0.104383         0.319778        0.342263        0.298716   1734   
6          0.100923         0.281226        0.298701        0.264851   1734   
5          0.080115         0.249594        0.264842        0.234574   1735   
4          0.060554         0.219586        0.234557        0.204605   1734   
3          0.042676         0.189934        0.204574        0.175417   1734   
2          0.042676         0.161562        0.175357        0.146771   1734   
1          0.026513         0.125786        0.146768        0.069447   1735   

        label_good  label_bad  
Decile                         
10            1237        498  
9             1413        321  
8             1509        225  
7             1553        181  
6             1559        175  
5             1596        139  
4             1629        105  
3             1660         74  
2             1660         74  
1             1689         46

In [40]:
#Save the model
xgb_model.save_model('LeoPayu_BBILAP_weightVintage_60_in_15mob.json')
print('Model Saved')

Model Saved
